# Load libraries

In [6]:
%load_ext autoreload
%autoreload 2


# Basic libraries
import pandas as pd
import numpy as np
import sys


# Scipy
import scipy
from scipy import signal
from scipy.linalg import solve
from scipy import constants
from scipy.interpolate import interp1d
from scipy.fft import fft, fftfreq

# Locate files
import os
from pathlib import Path
from glob import glob

# Plots
from matplotlib import pyplot as plt

#Important!! Make sure your current directory is the MHDTurbPy folder!
os.chdir("/Users/nokni/work/MHDTurbPy")


# Make sure to use the local spedas
sys.path.insert(0, os.path.join(os.getcwd(), 'pyspedas'))
import pyspedas
from pyspedas.utilities import time_string
from pytplot import get_data
from joblib import Parallel, delayed

""" Import manual functions """

sys.path.insert(1, os.path.join(os.getcwd(), 'functions'))
import calc_diagnostics as calc
import TurbPy as turb
import general_functions as func
import Figures as figs

from   SEA import SEA
import three_D_funcs as threeD
import download_data as download

sys.path.insert(1, os.path.join(os.getcwd(), 'functions/3d_anis_analysis_toolboox'))
import collect_wave_coeffs 
import data_analysis 

plt.rcParams['text.usetex'] = 0




os.environ["CDF_LIB"] = "/Applications/cdf/cdf/lib"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Download  WIND data

In [8]:
from joblib import Parallel, delayed

# If you only need to download 1 interval (dates wont matter if only_one_interval=0)

save_destination        =  f'/Users/nokni/work/MHDTurbPy/examples'


cdf_lib_path            = '/Applications/cdf/cdf/lib'            # You will need to read cdf files. You have to specify the path

#User defined parameters
credentials             =      { 'psp':{
                                           'fields': {'username': None, 'password': None},
                                           'sweap' : {'username': None, 'password': None}}}


settings                =  {
                            'Data_path'              : '/Volumes/Zesen-4TB/SPEDAS/' ,
                            'Only_1_interval'        :  True,                 # If you only want to check one interval
                            'start_date'             : '1995-04-11 00:00',
                            'end_date'               : '1995-05-11 00:00',
                            'overwrite_files'        :  1,                     # In case you want to re-do analysis for existing files!
                            'save_all'               :  True,                  # If false it will only keep derived quants. Not timeseries
                            'sc'                     : 'Ulysses',                 # Choices ['PSP', SOLO, 'HELIOS_A', 'HELIOS_B', 'WIND', 'Ulysses']
                            'part_resol'             : 3,                      # Max resol of plasma data                [s]
                            'MAG_resol'              : 1,                      # Max resol og magnetic field data        [s]

                            
                            'duration'               : '10H',                  # Duration of interval (in Hours)
                            'Step'                   : '5H',                   # Move starting point by step (in Hours)
                            'estimate_derived_param' :  True,
                            'Big_Gaps'               :{
                                                        'Mag_big_gaps' : 10,   # This will return 3 dtaframes containing gaps
                                                        'Par_big_gaps' : 20,   # in the respective timeseries
                                                        'QTN_big_gaps' : 300,
                                                        
                                                        },
                                                    
                            'cut_in_small_windows'   : {'flag'    : False, 
                                                        'Step'    : '10min',
                                                        'duration': '600min'},

                            'Max_par_missing'        : 10,
                            'addit_time_around'      : 4,                       # [start_time -addit_time_around [h], end_time + addit_time_around [h]]
                            'gap_time_threshold'     : 5 ,                      # Threshold for large gaps (units of seconds)
                            'apply_hampel'           : False,                   # Use hampelfilter to despike plasma data
                            'hampel_params'          : {'w'  : 200,
                                                        'std': 3},

                            'upsample_low_freq_ts'   : 0,                    #
                            'estimate_psd_b'         : 1,                       # Estimate magentic field powes spectral density (keep false)
                            'estimate_psd_v'         : 1,                       # Estimate velocity field powes spectral density (keep false)
                            'est_PSD_components'     : 1,
                            'smooth_psd'             : False,
                            'in_rtn'                 : True,                    # RTN or spacecraft frame ( We will usually use RTN)
                            'rol_mean'               : True,                    # To estimate fluctuations of mag, vel field
                            'rol_window'             : '1H',                    # When estimating fluctuations, size of window (centered around -w/2 -w/2

                            } 



if settings['sc'] == "PSP":
    vars_2_downnload = {
                        'mag'    : None, 
                        'span'   : None,
                        'span-a' : None,
                        'spc'    : None, 
                        'qtn'    : None,
                        'ephem'  : None}

elif settings['sc'] == "SOLO":
    vars_2_downnload = {
                        'mag'    : None,
                        'swa'    : None, 
                        'rpw'    : None, # Default is 'bia-density-10-seconds', but  'bia-density' is also available and probaly interesting
                        'ephem'  : None} 
else:
    
    vars_2_downnload = None
    

save_path = Path(save_destination).joinpath(settings['sc'])



generated_interval_list = download.generate_intervals(settings['start_date'],
                                                     settings['end_date'], 
                                                     settings            = settings,
                                                     data_path           = settings['Data_path'])


# Call function
Parallel(n_jobs=1)(delayed(download.download_files)(
                                                            jj, 
                                                            generated_interval_list, 
                                                            settings, 
                                                            vars_2_downnload, 
                                                            cdf_lib_path, 
                                                            credentials, 
                                                            save_path) for jj in range(len(generated_interval_list)))


17-May-24 17:34:38: Generating only one interval based on the provided start and end times.
17-May-24 17:34:38: Start Time: 1995-04-11 00:00:00
17-May-24 17:34:38: End Time: 1995-05-11 00:00:00
17-May-24 17:34:38: Considering an interval spanning: 1995-04-11 00:00:00 to 1995-05-11 00:00:00
17-May-24 17:34:38: Overwriting folder /Users/nokni/work/MHDTurbPy/examples/Ulysses/1995-04-11_00-00-00_1995-05-11_00-00-00_sc_0


Working on Ulysses data
Note this is NOT in RTN!!!


17-May-24 17:34:38: https://cdaweb.gsfc.nasa.gov/WS/cdasr/1/dataviews/sp_phys/datasets failed with http code 404
17-May-24 17:34:38: data_request = <cdasws.datarequest.CdfRequest object at 0x7fb4e1763880>
17-May-24 17:34:38: response.text: {"Message":["No data available."],"Warning":[],"Status":["No data found for HEL2_6SEC_NESSMAG, time range: 1995-04-10T20:00:00 - 1995-05-11T04:00:00"],"Error":["No data available."]}
17-May-24 17:34:38: get_data_result failed with http code 404
17-May-24 17:34:38: data_request = <cdasws.datarequest.CdfRequest object at 0x7fb4e1763880>
Traceback (most recent call last):
  File "/Users/nokni/work/MHDTurbPy/functions/downloading_helpers/Ulysses.py", line 240, in LoadTimeSeriesUlysses
    dfmag, dfmag1, infos =  LoadHighResMagUlysses(pd.Timestamp(t0),
  File "/Users/nokni/work/MHDTurbPy/functions/downloading_helpers/Ulysses.py", line 123, in LoadHighResMagUlysses
    index = data['Epoch'],
TypeError: 'NoneType' object is not subscriptable
17-May-24 17:34

failed at index 0 with error: local variable 'dfpar' referenced before assignment


Traceback (most recent call last):
  File "/Users/nokni/work/MHDTurbPy/functions/downloading_helpers/Ulysses.py", line 268, in LoadTimeSeriesUlysses
    dfpar, dfdis     = LoadTimeSeriesUlysses_particles(pd.Timestamp(t0),
  File "/Users/nokni/work/MHDTurbPy/functions/downloading_helpers/Ulysses.py", line 79, in LoadTimeSeriesUlysses_particles
    index = data['Epoch'],
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "/Users/nokni/work/MHDTurbPy/functions/downloading_helpers/Ulysses.py", line 314, in LoadTimeSeriesUlysses
    return diagnostics_MAG["resampled_df"].interpolate().dropna(), dfpar.interpolate().dropna(), dfdis, big_gaps, misc
KeyError: 'resampled_df'
Traceback (most recent call last):
  File "/Users/nokni/work/MHDTurbPy/functions/download_data.py", line 310, in main_function
    dfmag,  dfpar, dfdis, big_gaps, misc               = LoadTimeSeriesUlysses(start_time,
TypeError: cannot unpack non-iterable NoneType object
Traceback (mo

[None]